This analysis estimates how often an asset breaches a specified downside threshold over a configurable holding window (e.g., 1-week, 2-week, 3-week, monthly).

For each window, returns are computed end-to-end as:

Return = Price_end / Price_start − 1


The script reports the frequency with which returns fall below a chosen threshold (e.g., −2.5%, −5%, −9%). This provides a simple, strike-agnostic proxy for how often a breakeven level would historically be violated over different time horizons.

Results are purely price-based and intended as a first-pass downside risk sanity check for short-dated option strategies.

In [29]:
import pandas as pd
import yfinance as yf
import numpy as np

# -----------------------------
# Parameters (edit these)
# -----------------------------
ticker = "ELV"

# Timeline
USE_YEARS = False
years = 10

start_date = "2015-01-01"
end_date   = None  # None = today

# Threshold + window
threshold = -0.09          # e.g., -2.5%
WINDOW = "3W"               # "1W", "2W", "4W", "1M", "3M"
ANCHOR = "W-FRI"            # "W-FRI" (week ends Fri) or "W" etc.
RETURN_TYPE = "simple"      # "simple" or "log"

# -----------------------------
# Load daily data
# -----------------------------
if USE_YEARS:
    hist = yf.Ticker(ticker).history(period=f"{years}y", auto_adjust=True)
else:
    hist = yf.Ticker(ticker).history(start=start_date, end=end_date, auto_adjust=True)

required_cols = {"Close"}
if not required_cols.issubset(hist.columns):
    raise ValueError(f"Missing columns. Found: {list(hist.columns)}")

hist = hist[hist.index.dayofweek < 5].copy()  # defensive weekdays only

# -----------------------------
# Build windowed returns (end-to-end over the window)
# -----------------------------
# Resample to window endpoints (e.g., weekly Friday closes, 2W Friday closes, monthly closes)
close = hist["Close"].dropna()

# NOTE: "1M" in pandas is calendar month end. "4W" is 4-week blocks.
window_close = close.resample(WINDOW).last()

if RETURN_TYPE == "log":
    window_ret = np.log(window_close / window_close.shift(1))
else:
    window_ret = window_close / window_close.shift(1) - 1

window_ret = window_ret.dropna().rename("ret")

# -----------------------------
# Statistics
# -----------------------------
total_periods = len(window_ret)
count_breach = int((window_ret <= threshold).sum())
prob_breach = count_breach / total_periods if total_periods > 0 else np.nan

print(f"Ticker: {ticker}")
if USE_YEARS:
    print(f"Window: last {years} years")
else:
    print(f"Window: {start_date} to {end_date or 'today'}")

print(f"\nReturn horizon: {WINDOW}")
print(f"Total periods analyzed: {total_periods}")
print(f"Periods with return ≤ {threshold:.1%}: {count_breach} ({prob_breach:.2%})")

print("\nSummary of windowed returns:")
print(window_ret.describe(percentiles=[0.01, 0.05, 0.10]))

print(f"\nMost recent periods breaching {threshold:.1%}:")
print(
    window_ret[window_ret <= threshold]
    .tail(10)
    .to_frame()
    .assign(ret_pct=lambda x: (x["ret"] * 100).round(2))
)


Ticker: ELV
Window: 2015-01-01 to today

Return horizon: 3W
Total periods analyzed: 193
Periods with return ≤ -9.0%: 9 (4.66%)

Summary of windowed returns:
count    193.000000
mean       0.008027
std        0.063715
min       -0.206954
1%        -0.176576
5%        -0.089450
10%       -0.059212
50%        0.007639
max        0.213346
Name: ret, dtype: float64

Most recent periods breaching -9.0%:
                                ret  ret_pct
Date                                        
2018-12-23 00:00:00-05:00 -0.129177   -12.92
2019-04-28 00:00:00-04:00 -0.091746    -9.17
2020-03-29 00:00:00-04:00 -0.206954   -20.70
2020-06-21 00:00:00-04:00 -0.091242    -9.12
2020-09-13 00:00:00-04:00 -0.090220    -9.02
2021-02-07 00:00:00-05:00 -0.108970   -10.90
2024-11-03 00:00:00-04:00 -0.175082   -17.51
2025-07-13 00:00:00-04:00 -0.090382    -9.04
2025-08-03 00:00:00-04:00 -0.193765   -19.38


In [28]:

# -----------------------------
# USER INPUTS
# -----------------------------
TICKERS = ["SPY", "SGOL", "ELV", "USAR", "^VIX"]
START = "2020-01-01"
END = "2025-12-31"

# Choose: "D" = daily, "W" = weekly, "M" = monthly
FREQ = "D"

RETURN_TYPE = "log"   # "log" or "simple"

# -----------------------------
# DOWNLOAD PRICES
# -----------------------------
prices = (
    yf.download(TICKERS, start=START, end=END, auto_adjust=True)["Close"]
    .dropna()
)

# -----------------------------
# RESAMPLE PRICES
# -----------------------------
prices = prices.resample(FREQ).last()

# -----------------------------
# COMPUTE RETURNS
# -----------------------------
if RETURN_TYPE == "log":
    returns = np.log(prices / prices.shift(1))
else:
    returns = prices.pct_change()

returns = returns.dropna()

# -----------------------------
# CORRELATION MATRIX
# -----------------------------
corr_matrix = returns.corr()

display(corr_matrix)



[*********************100%***********************]  5 of 5 completed


Ticker,ELV,SGOL,SPY,USAR,^VIX
Ticker,,,,,
ELV,1.000000,0.020733,0.128851,0.040148,-0.127010
SGOL,0.020733,1.000000,0.144749,0.053998,-0.075201
SPY,0.128851,0.144749,1.000000,-0.021518,-0.823497
USAR,0.040148,0.053998,-0.021518,1.000000,-0.024539
^VIX,-0.127010,-0.075201,-0.823497,-0.024539,1.000000
